In [24]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from deeplab_mdl_def import DynamicUpsample

In [25]:
home_path = "/repo/tanfoni/"

parent_dir_train = home_path + "Dataset_merged/Train/"
real_dir_train = "Real"
sint_dir_train = "Fake"

parent_dir_val = home_path + "Dataset_merged/Valid/"
real_dir_val = "Real"
sint_dir_val = "Fake"

parent_dir_test = home_path + "Dataset_merged/Test/"
real_dir_test = "Real"
sint_dir_test = "Fake"

model_path = ('Results/'
              'Deeplab+fake_detection/trained_models/'
              'deeplab+fake_detection_transfer_learning_all_transfer_learning.h5') 
model = load_model(model_path, custom_objects={'DynamicUpsample': DynamicUpsample})

In [26]:
# Cell 3: Define data loading and augmentation parameters
train_batch_size = 32  # Adjust these parameters as needed
validation_batch_size = 32
test_batch_size = 32
resize_dim = (224, 224)  # Assuming 224x224 images for simplicity

# Data augmentation configuration
use_data_aug = True
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
) if use_data_aug else None



In [27]:
if use_data_aug:
    train_dataset = data_augmentation.flow_from_directory(
        parent_dir_train,
        target_size=resize_dim,
        batch_size=train_batch_size,
        class_mode='binary',
        shuffle=True,
        seed=123
    )

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    parent_dir_val,
    labels="inferred",
    class_names=[real_dir_val, sint_dir_val],
    label_mode='binary',
    color_mode='rgb',
    batch_size=validation_batch_size,
    image_size=resize_dim,
    shuffle=True,
    seed=123
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    parent_dir_test,
    labels="inferred",
    class_names=[real_dir_test, sint_dir_test],
    label_mode='binary',
    color_mode='rgb',
    batch_size=test_batch_size,
    image_size=resize_dim,
    shuffle=True,
    seed=123
)

Found 210000 images belonging to 2 classes.
Found 30000 files belonging to 2 classes.
Found 60000 files belonging to 2 classes.


In [28]:
n_fakes = len(os.listdir(os.path.join(parent_dir_train, sint_dir_train)))
n_reals = len(os.listdir(os.path.join(parent_dir_train, real_dir_train)))
total = n_fakes + n_reals
weight_for_0 = (1 / n_reals) * total / 2.0
weight_for_1 = (1 / n_fakes) * total / 2.0
class_weights = {0: weight_for_0, 1: weight_for_1}

In [29]:
test_loss, test_accuracy, test_precision, test_recall, test_weighted_accuracy = model.evaluate(test_dataset)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")


1875/1875 [==============================] - 53s 28ms/step - loss: 0.1692 - accuracy: 0.9342 - precision: 0.9509 - recall: 0.9505 - weighted_accuracy: 0.9342
Test Loss: 0.1691867709159851
Test Accuracy: 0.934249997138977
Test Precision: 0.9509016871452332
Test Recall: 0.9504500031471252
Test F1 Score: 0.9506757914952857
